## Custom Model

### Initialization

In [108]:
import json
import pandas as pd

### Import JSON 

In [142]:
df = pd.read_json('intents.json')
df['text'] = df['intents'].apply(lambda x: x['dialogue'])
df['intent'] = df['intents'].apply(lambda x: x['intent'])
df.drop('intents', axis=1, inplace=True)
df.head()

f = open('intents.csv', 'w')
df['text'].to_csv(f, index=False)

### Drop Duplicates

In [110]:
df.drop_duplicates(subset='text', inplace=True)

### Value Counts

In [111]:
df['intent'].value_counts()

intent
History         36
Book            34
Cancel          32
Location        32
About           29
Availability    26
Name: count, dtype: int64

### Embeddings


In [112]:
from transformers import BertTokenizer,TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model = TFBertModel.from_pretrained('bert-base-uncased')
df['tokenized'] = df['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, return_tensors='tf'))
df['embeddings'] = [model(token).pooler_output for token in df['tokenized']]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [113]:
correspondence = {'History':0,'Book':1,'Cancel':2,'Location':3,'About':4,'Availability':5}
reversed_correspondence = {v:k for k,v in correspondence.items()}
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
df['intent'] = df['intent'].replace(correspondence, inplace=False)
X = df['embeddings'].values
y = df['intent'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = np.concatenate([tensor.numpy() for tensor in X_train])
X_test = np.concatenate([tensor.numpy() for tensor in X_test])

C:\Users\Rishabh\AppData\Local\Temp\ipykernel_26476\3622762613.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['intent'] = df['intent'].replace(correspondence, inplace=False)


In [115]:
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.callbacks import EarlyStopping

callback = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model = Sequential()
model.add(Dense(512, input_dim=768, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='sigmoid')) 
model.add(Dropout(0.5))
model.add(Dense(128, activation='sigmoid')) 
model.add(Dense(1, activation='softmax'))


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
hist = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[callback])
# X_train.shape

Epoch 1/100
5/5 [==============================] - 1s 47ms/step - loss: 1.0813 - accuracy: 0.1921 - val_loss: -2.3746 - val_accuracy: 0.1316
Epoch 2/100
5/5 [==============================] - 0s 19ms/step - loss: -3.0029 - accuracy: 0.1921 - val_loss: -5.0645 - val_accuracy: 0.1316
Epoch 3/100
5/5 [==============================] - 0s 17ms/step - loss: -5.2888 - accuracy: 0.1921 - val_loss: -6.8571 - val_accuracy: 0.1316
Epoch 4/100
5/5 [==============================] - 0s 16ms/step - loss: -6.9312 - accuracy: 0.1921 - val_loss: -8.0730 - val_accuracy: 0.1316
Epoch 5/100
5/5 [==============================] - 0s 15ms/step - loss: -7.9549 - accuracy: 0.1921 - val_loss: -8.9317 - val_accuracy: 0.1316
Epoch 6/100
5/5 [==============================] - 0s 31ms/step - loss: -8.9319 - accuracy: 0.1921 - val_loss: -9.5702 - val_accuracy: 0.1316
Epoch 7/100
5/5 [==============================] - 0s 20ms/step - loss: -9.5222 - accuracy: 0.1921 - val_loss: -10.1139 - val_accuracy: 0.1316
Epoch 

In [136]:
from fuzzywuzzy import fuzz
dictionary = {'intent':[],'combined_text':[],'similarity':[]}
for i in range(6):
    dictionary['intent'].append(i)
    dictionary['combined_text'].append(' '.join(df[df['intent']==i]['text'].values))
    dictionary['similarity'].append(np.nan)
df_new = pd.DataFrame(dictionary)
df_new['intent'].replace(reversed_correspondence,inplace=True)
name = input('Enter text: ')
df_new['similarity'] = df_new['combined_text'].apply(lambda x: fuzz.partial_ratio(x,name))
df_new

C:\Users\Rishabh\AppData\Local\Temp\ipykernel_26476\1891102389.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_new['intent'].replace(reversed_correspondence,inplace=True)


,intent,combined_text,similarity
0,History,What is the history of the temple? History Can...,39
1,Book,How to book ticket? Is it possible to schedule...,29
2,Cancel,"Cancel a ticket Hi, I need to cancel my upcomi...",32
3,Location,Where is the temple located? Location Can you ...,39
4,About,Tell me about the temple What is the signific...,32
5,Availability,Any tickets available from 15th April to 20th ...,29


In [141]:
import spacy
name = input('Enter text: ')
nlp = spacy.load('en_core_web_sm')
for i in range(6):
    print(df_new.iloc[i]['intent'], " -> ",nlp(name).similarity(nlp(df_new.iloc[i]['combined_text'])))

C:\Users\Rishabh\AppData\Local\Temp\ipykernel_26476\1703778463.py:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  print(df_new.iloc[i]['intent'], " -> ",nlp(name).similarity(nlp(df_new.iloc[i]['combined_text'])))


History  ->  0.564681918849925
Book  ->  0.5526458447059821
Cancel  ->  0.4572201926491917
Location  ->  0.6114424142237456
About  ->  0.5695678889446457
Availability  ->  0.4602208883740392


In [149]:
for row in df[df['intent']=='About']['text'].values:
    print(row)

Tell me about the temple
 What is the significance of this mandir?
Can you tell me a story about the mandir?
Is there anything special about the architecture of this mandir?
I'm curious about the mandir, can you tell me something about it?
I would be grateful if you could share information on the temple's architectural style and significant features.
The temple's deities are of particular interest to me. Could you elaborate on them?
Is there a website or brochure available with a detailed description of the temple layout and facilities?
I would appreciate any insights you can offer regarding the temple's daily rituals and ceremonies.
Could you outline the key structures and shrines located within the temple grounds?
If available, I would be interested in learning about any festivals or special events unique to the temple.
For planning purposes, are there specific opening and closing hours for the temple?
Does the temple have any specific dress code requirements for visitors?
I would ap